In [ ]:
# Installazione delle librerie necessarie (eseguire una volta sola se necessario)
# %pip install pandas numpy matplotlib seaborn scikit-learn

# Esercitazione Machine Learning - AutomaParts S.p.A.

In questo notebook andremo a sviluppare un modello di machine learning per l'azienda **AutomaParts S.p.A.**.
L'obiettivo è prevedere se un pezzo prodotto è difettoso (defect_label = 1) o conforme (defect_label = 0) basandoci sulla varie misure rilevate durante la produzione.

## 1. Importazione delle librerie
Iniziamo importando le librerie necessarie per l'analisi e la modellazione.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerie per il machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## 2. Caricamento e Analisi dei Dati
Carichiamo il dataset `parts_production_data.csv` e diamo un'occhiata alle prime righe per capire come è fatto.

In [ ]:
df = pd.read_csv('parts_production_data.csv')
df.head()

In [ ]:
print("Dimensione del dataset:", df.shape)
df.info()

Vediamo un po' di statistiche descrittive per le colonne numeriche.

In [ ]:
df.describe()

Da questa tabella vediamo subito i valori medi, i minimi e i massimi. Ci serve per capire se ci sono valori "strani" (tipo una temperatura troppo alta o troppo bassa) e come sono distribuiti i dati nelle varie misure.

### Analisi della variabile target
Controlliamo quanti pezzi sono difettosi e quanti no. È importante vedere se le classi sono bilanciate.

In [ ]:
print(df['defect_label'].value_counts())
sns.countplot(x='defect_label', data=df)
plt.title('Distribuzione Difetti')
plt.show()

Come possiamo vedere, c'è uno sbilanciamento (ci sono meno difetti che pezzi buoni), il che è normale, altrimenti sarebbe una linea produttiva assai disastrosa.

### Matrice di correlazione
Vediamo se ci sono variabili molto correlate tra loro o con il target.

In [ ]:
plt.figure(figsize=(14, 10)) # Dimesioni più o meno decenti per leggere i numeri internin
sns.heatmap(
    df.corr(numeric_only=True), 
    annot=True, 
    fmt=".2f",           # Solo 2 decimale per risparmiare spazio (0.9 invece di 0.92)
    annot_kws={"size": 9}, # Font più piccoli
    cmap='coolwarm'
)

Dalla matrice di correlazione possiamo notare alcune cose interessanti. Ad esempio, si vede che alcune variabili hanno un quadratino più colorato vicino alla nostra 'defect_label'. Questo significa che quelle misure (come magari il diametro o il punteggio dell'ispezione visiva) sono più legate al fatto che un pezzo sia difettoso o meno. 

Se il numero è vicino a 1 o -1 c'è molta correlazione, se è vicino a 0 quasi per niente. Mi sembra che il punteggio visivo e forse la temperatura abbiano un peso, ma poi lo vedremo meglio con i modelli.

Qui vediamo chiaramente che ci discostiamo poco dallo 0 un po' per tutte le colonne, quindi non si evidenza una correlazione diretta.

## 3. Pulizia e Preparazione dei Dati (Data Cleaning)

Adesso controlliamo se ci sono valori mancanti (NaN).

In [ ]:
df.isnull().sum()

Come possiamo vedere non ci sono colonne/feature con valori mancanti nei sample/righe. Un dataset gia molto valido.

In [ ]:
# Riempiamo i valori nulli con la media solo per le colonne numeriche
colonne_numeriche = df.select_dtypes(include=[np.number]).columns
df[colonne_numeriche] = df[colonne_numeriche].fillna(df[colonne_numeriche].mean())

# Ricontrollo
df.isnull().sum().sum()

Il risultato 0 ci conferma che non abbiamo più valori nulli nel dataset. Adesso possiamo pulire il dataframe rimuovendo le colonne che non ci servono per il modello.

Eliminiamo le colonne che non servono per la predizione, come `part_id` (è solo un codice) e `production_timestamp` (per ora non facciamo analisi temporali complesse).

In [ ]:
df = df.drop(columns=['part_id', 'production_timestamp'], errors='ignore')
df.head()

### Gestione Variabili Categoriche
Abbiamo colonne come `line_id`, `station_id` ecc. che sono categorie. Le trasformiamo in numeri usando il `LabelEncoder` o `get_dummies`. Qui uso `get_dummies` per semplicità sulle variabili con poche categorie, e `LabelEncoder` per i batch se sono troppi, ma facciamo tutto con `get_dummies` per fare prima, oppure LabelEncoder se sono tante uniche.

In [ ]:
# Vediamo quante categorie uniche ci sono
print("Batch unici:", df['material_batch'].nunique())
print("Line ID unici:", df['line_id'].nunique())

Ci sono quasi 3000 batch diversi ma solo 10 linee! Se usassimo 'get_dummies' sui batch verrebbe fuori una tabella gigante con 3000 colonne, meglio usare il LabelEncoder altrimenti ci mettiamo troppo tempo. Per le linee invece 10 sono poche, potremmo quasi lasciarle così o usare dummies.

Visto che `material_batch` ha molte varianti, usiamo LabelEncoder per quella, e get_dummies per le altre (che però in questo dataset sono già numeriche o quasi, `line_id` è numerico ma rappresenta una categoria).

In [ ]:
# Converto measure e altre se necessario. Ma le colonne ID sembrano numeri interi, le trattiamo come numeri o categorie?
# Per semplicità le lascio come numeri per ora, ma material_batch è stringa.

le = LabelEncoder()
df['material_batch'] = le.fit_transform(df['material_batch'])

Perfetto, ora che abbiamo trasformato i codici dei batch in numeri, il modello potrà usarli senza problemi. Abbiamo mantenuto le altre variabili come numeri perché sembrano già codificate correttamente.

### Scaling delle Features
Le variabili hanno scale diverse (es. diametro in mm, temperatura in gradi). Meglio portarle tutte sulla stessa scala.

In [ ]:
# Divido in X e y
X = df.drop('defect_label', axis=1)
y = df['defect_label']

# Divido in Training e Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set:", X_train.shape)
print("Test set:", X_test.shape)

Qui abbiamo diviso i dati: l'80% lo usiamo per 'insegnare' al modello e il 20% lo teniamo da parte per vedere se ha imparato bene o se ha solo imparato a memoria (quello che si chiama overfitting).

In [ ]:
scaler = StandardScaler()

# Adatto lo scaler solo sul train per evitare data leakage
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

E qui abbiamo usato lo StandardScaler. Non è proprio una 'normalizzazione' classica (quella 0-1), ma una 'standardizzazione'. Praticamente schiaccia tutti i numeri in modo che la media sia 0. Così il modello non si confonde se una variabile ha numeri giganti (come la temperatura) e un'altra piccolissimi (come il diametro).

## 4. Modellazione
Proveremo tre modelli diversi come richiesto:
1. Logistic Regression
2. Decision Tree
3. Random Forest

### Modello 1: Logistic Regression

In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train_scaled, y_train)

y_pred_log = log_reg.predict(X_test_scaled)

print("Accuracy Logistic Regression:", accuracy_score(y_test, y_pred_log))

Niente male come inizio! La regressione logistica ci dà un'accuratezza del 75% circa. Come modello base è solido, anche se forse un po' troppo semplice per catturare tutte le sfumature della produzione.

### Modello 2: Decision Tree
Gli alberi decisionali sono facili da interpretare.

In [ ]:
# Non serve scaling per i tree based solitamente, ma usiamo X_train normale
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

y_pred_tree = tree_clf.predict(X_test)

print("Accuracy Decision Tree:", accuracy_score(y_test, y_pred_tree))

L'albero decisionale è sceso un po', siamo intorno al 68%. Probabilmente un solo albero fa fatica a generalizzare bene su questi dati, o magari è andato un po' in crisi con qualche variabile.

### Modello 3: Random Forest
Un insieme di alberi, di solito più robusto.

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)

print("Accuracy Random Forest:", accuracy_score(y_test, y_pred_rf))

Ecco, il Random Forest è tornato su, superando il 78%. Mettere insieme tanti alberi aiuta quasi sempre a correggere gli errori dei singoli alberi, confermandosi il modello più robusto per questo tipo di problema.

## 5. Valutazione e Confronto

Confrontiamo i risultati dei tre modelli usando metriche più dettagliate come la Confusion Matrix e il Classification Report.

In [ ]:
print("--- Logistic Regression ---")
print(classification_report(y_test, y_pred_log))

print("\n--- Decision Tree ---")
print(classification_report(y_test, y_pred_tree))

print("\n--- Random Forest ---")
print(classification_report(y_test, y_pred_rf))

Analizzando bene i report qui sopra, notiamo una cosa molto interessante (e un po' preoccupante):


La **Logistic Regression** ha un accuracy del 76%, ma se guardiamo bene non ha beccato *nemmeno un pezzo difettoso* (precision e recall per la classe 1 sono a zero!). Praticamente ha fatto la 'pazza' e ha detto che tutti i pezzi sono buoni. Siccome la maggior parte dei pezzi lo è davvero, l'accuratezza sembra alta, ma il modello è inutile per noi.

Il **Decision Tree** invece, pur avendo un accuracy totale più bassa (69%), ha iniziato a 'vedere' i difetti, con una recall del 39%. Almeno ci prova!

Il **Random Forest** è il vincitore perché tiene insieme le due cose: ha l'accuratezza più alta e riesce anche a scovare i pezzi difettosi meglio degli altri.

Andiamo a vedere la matrice di confusione per il modello migliore (probabilmente il Random Forest).

In [ ]:
cm = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predetto')
plt.ylabel('Reale')
plt.title('Confusion Matrix Random Forest')
plt.show()

Guardando i numeri del Random Forest, ecco la situazione reale:

  * I pezzi "buoni" sono quasi tutti salvi: Il modello è fenomenale nel riconoscere i pezzi conformi. Ne ha azzeccati 450 e ha dato solo 4 "falsi allarmi" (pezzi buoni scambiati per difettosi).
  * Il vero problema sono i "falsi negativi": Qui c'è il tasto dolente. Ci sono ben 127 pezzi difettosi che il modello ha scambiato per buoni. Per un'azienda, questi sono i più pericolosi perché finiscono dritti al cliente!
  * Piccoli passi avanti: Rispetto alla Regressione Logistica (che ne prendeva zero), qui almeno 19 difetti li abbiamo intercettati.

In sintesi: il modello è molto prudente. Non sbaglia quasi mai a dare la colpa a un pezzo buono, ma si lascia sfuggire ancora troppi difetti. Potremmo provare a "registrarlo" meglio, magari abbassando la soglia di decisione per essere più severi!

### Feature Importance
Vediamo quali variabili hanno influito di più sulla decisione del modello.

In [ ]:
importances = rf_clf.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(10,6))
plt.title("Feature Importance (Random Forest)")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.show()

### Cosa ci dicono queste colonne?

Il grafico della **Feature Importance** ci svela quali sono i parametri che "muovono l'ago della bilancia" per il modello:

1.  **I fattori dominanti:** Le prime barre (che solitamente sono variabili come il punteggio dell'ispezione visiva, il diametro o la temperatura) sono quelle che il modello guarda con più attenzione. Se queste misure variano, è quasi certo che cambi la previsione del difetto.
2.  **Efficienza dei controlli:** Quelle in fondo alla classifica sono meno rilevanti. Sapere questo ci permette di capire che, se dovessimo risparmiare tempo sui controlli, potremmo concentrarci solo sulle variabili più "pesanti" senza perdere troppa precisione.
3.  **Focus sulla linea:** Spesso le variabili più importanti sono legate a fasi specifiche della produzione. Questo grafico ci dice dove la variabilità del processo crea più problemi di qualità e dove dovremmo intervenire per migliorare i macchinari!

## Conclusioni

Dai risultati ottenuti vediamo che il **Random Forest** (o il modello che ha performato meglio) è il più affidabile.
L'analisi ha mostrato che alcune variabili come il diametro e la temperatura sono molto importanti per predire i difetti.

Per AutomaParts S.p.A., utilizzare questo modello in produzione potrebbe permettere di intercettare molti pezzi difettosi prima che arrivino al cliente, riducendo i costi.
Bisogna però fare attenzione ai "Falsi Negativi" (pezzi difettosi predetti come buoni), perché quelli sono i più pericolosi. Magari si potrebbe abbassare la soglia di probabilità per essere più cautelativi.